In [ ]:
import selenium.webdriver
from bs4 import BeautifulSoup
import pandas as pd
import re
from time import sleep
import json
import csv
import requests

In [ ]:
driver = selenium.webdriver.Chrome(executable_path="C:/Users/janva/Downloads/chromedriver.exe")
driver.get("https://www.gall.nl/grolsch-lentebok-30cl-992739.html")

In [ ]:
res = driver.page_source.encode('utf-8')
soup = BeautifulSoup(res, "html.parser")

In [ ]:
def get_soup(url):
    #driver = selenium.webdriver.Chrome(executable_path="C:/Users/janva/Downloads/chromedriver.exe")
    #driver.get(url)
    #res = driver.page_source.encode('utf-8')
    #soup = BeautifulSoup(res, "html.parser")
    r = requests.get(url, headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"})
    soup = BeautifulSoup(r.text, "html.parser")
    return soup

In [ ]:
soup = get_soup("https://www.gall.nl/grolsch-lentebok-30cl-992739.html")

In [ ]:
r = requests.get("https://www.gall.nl/grolsch-lentebok-30cl-992739.html", headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"})
soup = BeautifulSoup(r.text, "html.parser")
soup

In [ ]:
soup = get_soup("https://www.gall.nl/lawsons-dry-hills-riesling-wit-75cl-121304.html")

In [ ]:
if "url" in json.loads(str(soup.find('script', {"type":"application/ld+json"}).contents[0].replace('\n', '')))[0]:
    print("yes")
else:
    print('no')

In [ ]:
data = []
reviews = json.loads(str(soup.find('script', {"type":"application/ld+json"}).contents[0].replace('\n', '')))[0]
if "review" in reviews:
    for review in reviews['review']:
        info = {'link' : "asdf",
                "rating" : review['reviewRating']['ratingValue'],
                "date" : review['datePublished'],
                "review" : review['description']}
        try:
            info['type'] = soup.find('span', class_="pdp-info_profile").text.strip()
        except:
            info['type'] = None
        try:
            info['rating_average'] = float(soup.find('span', class_ ="rating_label").text)
        except:
            info['rating_average'] = None
        try:
            info['count'] = int(soup.find('span', class_ ="rating_label")['data-count'])
        except:
            info['count'] = None
        try:
            info['title'] = soup.find('h1', class_ ="pdp-info_name").text
        except:
            info['title'] = None
        try:
            info['country'] = soup.find('span', class_ = re.compile("flag-round u-flag-large u-hidden"))['title']
        except:
            info['country'] = None
        try:
            info['size'] = soup.find('p', class_ ="pdp-info_desc").find_all('i')[0].text
        except:
            info['size'] = None
        try:
            info['percentage'] = soup.find('p', class_ ="pdp-info_desc").find_all('i')[1].text
        except:
            info['percentage'] = None
        try:
            info['year'] = soup.find('p', class_ ="pdp-info_desc").find_all('span')[2].text
        except:
            info['year'] = None
        try:
            info['badge'] = soup.find('p', class_ ="pdp-info_badges").text.strip()
        except:
            info['badge'] = None
        try:
            info['price'] = soup.find('strong', class_ = "price-v2 pdp-info_price base-price")['aria-label'].replace('€ ', '')
        except:
            info['price'] = None
        try:
            info['main_description'] = soup.find('div', class_ ="pdp-info_short s-rich-text").text.strip()
        except:
            info['main_description'] = None
        try:
            usps = []
            for item in soup.find('ul', class_ = "m-usp-bar").find_all('div', class_="usp__text"):
                usps.append(item.text.strip())
            info['usps'] = usps
        except:
            info['usps'] = None

        try:
            info['secondary_description'] = soup.find('div', class_ = "pdp-details-description s-rich-text").text.strip()
        except:
            info['secondary_description'] = None
        details = [e.text for e in soup.find_all('span', class_ ="pdp-details__item-label")]
        try:
            pos = details.index("Smaak")
            info['taste'] = soup.find_all('span', class_ ="pdp-details__item-label-secondary")[pos].text.replace('\n', '')
        except:
            info['taste'] = None
        try:
            pos = details.index("Lekker bij")
            info['complementaries'] = soup.find_all('span', class_ ="pdp-details__item-label-secondary")[pos].text.replace('\n', '')
        except:
            info['complementaries'] = None
        try:
            pos = details.index("Druivensoort")
            info['grape'] = soup.find_all('span', class_ ="pdp-details__item-label-secondary")[pos].text.replace('\n', '')
        except:
            info['grape'] = None
        try:
            pos = details.index("Etiketinformatie")
            info['label'] = soup.find_all('span', class_ ="pdp-details__item-label-secondary")[pos].text.replace('\n', '')
        except:
            info['label'] = None
        try:
            pos = details.index("Prijswinnend")
            info['certificate'] = soup.find_all('span', class_ ="pdp-details__item-label-secondary")[pos].text.replace('\n', '')
        except:
            info['certificate'] = None

        try:
            info['producer_description'] = soup.find('div', class_="background-story__description s-rich-text read-more__text").text.strip()
        except:
            info['producer_description'] = None
        try:
            info['producer'] = soup.find('span', class_="background-story__name").text
        except:
            info['producer'] = None
        data.append(info)
else:
    pass
data

In [ ]:
info.keys()

In [ ]:
soup.find('p', class_ ="pdp-info_desc").find_all('span')

In [ ]:
with open("C:/Users/janva/Documents/Git projects/gall_thesis2/gen/input/no_reviews.csv", "w", encoding = 'utf-8-sig', newline = '') as csv_file:
    writer = csv.writer(csv_file, delimiter = ";")
    writer.writerow(['link'])

In [ ]:
def get_product_info(product_url, data):
    soup = get_soup(product_url)
    reviews = json.loads(str(soup.find('script', {"type":"application/ld+json"}).contents[0].replace('\n', '')))[0]
    #Check if there are reviews
    if "review" in reviews:
        for review in reviews['review']:
            info = {'link' : product_url,
                    "rating" : review['reviewRating']['ratingValue'],
                    "date" : review['datePublished'],
                    "review" : review['description']}
            try:
                info['type'] = soup.find('span', class_="pdp-info_profile").text.strip()
            except:
                info['type'] = None
            try:
                info['rating_average'] = float(soup.find('span', class_ ="rating_label").text)
            except:
                info['rating_average'] = None
            try:
                info['count'] = int(soup.find('span', class_ ="rating_label")['data-count'])
            except:
                info['count'] = None
            try:
                info['title'] = soup.find('h1', class_ ="pdp-info_name").text
            except:
                info['title'] = None
            try:
                info['country'] = soup.find('span', class_ = re.compile("flag-round u-flag-large u-hidden"))['title']
            except:
                info['country'] = None
            try:
                info['size'] = soup.find('p', class_ ="pdp-info_desc").find_all('i')[0].text
            except:
                info['size'] = None
            try:
                info['percentage'] = soup.find('p', class_ ="pdp-info_desc").find_all('i')[1].text
            except:
                info['percentage'] = None
            try:
                info['year'] = soup.find('p', class_ ="pdp-info_desc").find_all('span')[2].text
            except:
                info['year'] = None
            try:
                info['badge'] = soup.find('p', class_ ="pdp-info_badges").text.strip()
            except:
                info['badge'] = None
            try:
                info['price'] = soup.find('strong', class_ = "price-v2 pdp-info_price base-price")['aria-label'].replace('€ ', '')
            except:
                info['price'] = None
            try:
                info['main_description'] = soup.find('div', class_ ="pdp-info_short s-rich-text").text.strip()
            except:
                info['main_description'] = None
            try:
                usps = []
                for item in soup.find('ul', class_ = "m-usp-bar").find_all('div', class_="usp__text"):
                    usps.append(item.text.strip())
                info['usps'] = usps
            except:
                info['usps'] = None

            try:
                info['secondary_description'] = soup.find('div', class_ = "pdp-details-description s-rich-text").text.strip()
            except:
                info['secondary_description'] = None
            details = [e.text for e in soup.find_all('span', class_ ="pdp-details__item-label")]
            try:
                pos = details.index("Smaak")
                info['taste'] = soup.find_all('span', class_ ="pdp-details__item-label-secondary")[pos].text.replace('\n', '')
            except:
                info['taste'] = None
            try:
                pos = details.index("Lekker bij")
                info['complementaries'] = soup.find_all('span', class_ ="pdp-details__item-label-secondary")[pos].text.replace('\n', '')
            except:
                info['complementaries'] = None
            try:
                pos = details.index("Druivensoort")
                info['grape'] = soup.find_all('span', class_ ="pdp-details__item-label-secondary")[pos].text.replace('\n', '')
            except:
                info['grape'] = None
            try:
                pos = details.index("Etiketinformatie")
                info['label'] = soup.find_all('span', class_ ="pdp-details__item-label-secondary")[pos].text.replace('\n', '')
            except:
                info['label'] = None
            try:
                pos = details.index("Prijswinnend")
                info['certificate'] = soup.find_all('span', class_ ="pdp-details__item-label-secondary")[pos].text.replace('\n', '')
            except:
                info['certificate'] = None

            try:
                info['producer_description'] = soup.find('div', class_="background-story__description s-rich-text read-more__text").text.strip()
            except:
                info['producer_description'] = None
            try:
                info['producer'] = soup.find('span', class_="background-story__name").text
            except:
                info['producer'] = None
            try:
                data.append(info)
                if len(data) % 100 == 0:
                    print(len(data))
            except:
                print('could not append')

            with open("C:/Users/janva/Documents/Git projects/gall_thesis2/gen/input/data_gall_with_reviews_corrected.csv", "a", encoding = 'utf-8-sig', newline = '') as csv_file:
                writer = csv.writer(csv_file, delimiter = ";")
                writer.writerow(list(info.values()))
    #Else there is no written review
    else:
        print(f"There are no reviews for {product_url}")
        with open("C:/Users/janva/Documents/Git projects/gall_thesis2/gen/input/no_reviews.csv", "a", encoding = 'utf-8-sig', newline = '') as csv_file:
            writer = csv.writer(csv_file, delimiter = ";")
            writer.writerow([product_url])
        sleep(2)


In [ ]:
data

In [ ]:
reviews = json.loads(str(soup.find('script', {"type":"application/ld+json"}).contents[0].replace('\n', '')))

In [ ]:
for review in reviews[0]['review']:
    info = {
    "rating" : review['reviewRating']['ratingValue'],
    "date" : review['datePublished'],
    "description" : review['description']}
    print(info)

In [ ]:
data = []
get_product_info("https://www.gall.nl/valdivieso-winemakers-reserva-cabernet-sauvignon-rood-75cl-387568.html", data)
data

In [ ]:
product_pages = pd.read_csv("C:/Users/janva/Documents/Git projects/gall_thesis2/gen/input/gall_product_links.csv", sep = ";", index_col = 0)
product_pages

In [ ]:
no_reviews = pd.read_csv("C:/Users/janva/Documents/Git projects/gall_thesis2/gen/input/no_reviews.csv", sep = ";")
no_reviews

In [ ]:
data_so_far = pd.read_csv("C:/Users/janva/Documents/Git projects/gall_thesis2/gen/input/data_gall_with_reviews_corrected.csv", sep = ";")
data_so_far
links_so_far = data_so_far['link'].unique()
links_to_do = [link for link in product_pages['product_link'] if link not in links_so_far]
print(len(product_pages))
print(len(links_so_far))
print(len(links_to_do))
print(len(no_reviews))

In [ ]:
data = []
i = 0
#driver = selenium.webdriver.Chrome(executable_path="C:/Users/janva/Downloads/chromedriver.exe")
for product_link in links_to_do:
    if product_link not in list(no_reviews['link']):
        print(product_link)
        if i % 100 == 0:
            print(round(i/len(links_to_do),4)*100)
        i += 1
        try:
            get_product_info(product_link, data)
        except:
            print(f"unknown error for {product_link}")
            with open("C:/Users/janva/Documents/Git projects/gall_thesis2/gen/input/no_reviews.csv", "a", encoding = 'utf-8-sig', newline = '') as csv_file:
                writer = csv.writer(csv_file, delimiter = ";")
                writer.writerow([product_link])
    else:
        print(f"already looked at {product_link}, there are no reviews")

# Concationation

In [ ]:
beer = pd.read_csv("C:/Users/janva/Documents/Git projects/gall_thesis2/gen/input/data_beergall_with_reviews_corrected.csv", sep = ";")
wine = pd.read_csv("C:/Users/janva/Documents/Git projects/gall_thesis2/gen/input/data_gall_with_reviews_corrected.csv", sep = ";")

In [ ]:
beer['category'] = "beer"
wine['category'] = "wine"

In [ ]:
print(len(wine))
print(len(beer))
concat_data = pd.concat([wine,beer])

In [ ]:
concat_data.to_csv("C:/Users/janva/Documents/Git projects/gall_thesis2/gen/input/full_data_set.csv", sep = ";")